<a href="https://colab.research.google.com/github/soham-malhotra/Library_Database_Project/blob/main/DatabaseProjectLibrary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Management System

---

### Introduction
This project aims to create a database-driven Library Management System (LMS) to manage books, borrowers, and transactions efficiently. The database will allow librarians to track the availability of books, manage borrower details, and log the issuance and return of books.

### Goals
To develop an easy-to-manage database system to keep track of:

*   ***Books*** (title, author, genre, availability, etc.)
*   ***Borrowers*** (name, contact info, borrowed books, etc.)
*   ***Transactions*** (check-out, return, overdue fines)


### What were the key relationships we focused on


*  A borrower can borrow multiple books unless he has an overdue book
*  A book can be borrowed by one borrower at a time
*  If the borrower has the book for more than 15 days, and someone requests it,
he must return it
*  Each transaction logs a book’s borrowing and returning.

---

##Imports





















In [ ]:
# for the database
import pickle as pkl
import os
import polars as pl

#for the interactivity
import ipywidgets as widgets
from IPython.display import display

#for sanity checks of book info
import re

We add the necessary imports to create the database, for which we use polars and this database is then serialized and stored/accessed using pickle


In [ ]:
# Define the Book class
class Book:
    # initialise Book with all necessary components
    def __init__(self, title, author, isbn):
        self.title = title
        self.author = author
        self.isbn = isbn

    # print Book info (e.g. Harry Potter The Goblet of Fire by JK Rowling, ISBN: 0-7475-5079-4)
    def __repr__(self):
      return f"{self.title} by {self.author}, ISBN : {self.isbn}"

class Borrower:
    #initialise Borrower with all necessary components
    def __init__(self, userID, borrowed_books, email):
      self.userID = userID
      self.borrowed_books = borrowed_books
      self.email = email

    #print user info
    def __repr__(self):
      return f"User : {self.userID} Contact Information: {self.email}\n Borrowed Books : {self.borrowed_books}"


Now we have defined the Book class, Borrower class with all their required components. Nowe we can create the Library class which is the main component of the project, with functions on database manipulation and usage using these classes


In [ ]:

class Library:
    #initialise all necessary components
    def __init__(self, filename_books='library_books.pkl', filename_users='library_users.pkl'):
        self.filename_books = filename_books
        self.filename_users = filename_users
        self.books = self.load_database(filename_books) # Map[Books, Int] : The book mapped to quantity in library
        self.users = self.load_database(filename_users) # Map[String, Borrower]

    # Load books from file
    def load_database(self, filename):
        if os.path.exists(filename):
            with open(filename, 'rb') as file:
                return pickle.load(file)
        else:
            return {}

    # Get all books in database
    def view_all(self):
      if not self.books:
        print(f"ANARCHYYYY")
      else:
        for book, quantity in self.books.items():
          print(book, "\n", f"Quantity: {quantity}\n")
          print(' '*10 + '*'*20 + ' '*10, "\n") # empty line

    # Creates a user
    def create_user(self):
      userID = input("Please enter a userID: ")
      email = input("Please enter your email: ")
      return Borrower(userID,[],email)


    # returns user or creates it if not there
    def get_user(self, borrower_name):
      if borrower_name in self.users:
        return self.users[borrower_name]
      else:
        borrower = self.create_user()
        self.users[borrower_name] = borrower
        self.update_database("users")
        return borrower


     # updated database, option is either "books" or "users"
    def update_database(self, option):
      if option == "books":
        with open(self.filename_books, 'wb') as file:
            pickle.dump(self.books, file)
      elif option == "users":
        with open(self.filename_users, 'wb') as file2:
            pickle.dump(self.users,file2)


     # check if book exists / if its in stock
    def book_exists(self, book_title):
     # Check if any book in self.books has a title that matches book_title
      for book in self.books.keys():
          if book.title == book_title and self.books[book] != 0:
            return True
      return False


    # checks if string has special char or num
    def contains_special_chars_or_numbers(self, string):
      pattern = r'[^a-zA-Z]'
      return bool(re.search(pattern, string))

    # check if isbn is valid
    def valid_isbn(self,isbn):
      return all(c.isdigit() or c == 'X' for c in isbn)


     # Add a book (in case of donation or return for example)
    def add_book(self, borrower_name, book_title, book_author, book_isbn):
      #sanity check
      if(self.contains_special_chars_or_numbers(book_author) or not self.valid_isbn(book_isbn)):
        print(f"aborted as incorrect data format, please check correct ISBN10 or author name")
        return
      #check it exists
      book = Book(book_title, book_author, book_isbn)
      if self.book_exists(book_title):
        print(self.books)
        print(self.books[book])
        self.books[book] += 1
        borrower = self.get_user(borrower_name)
        borrower.borrowed_books.remove(book)
        print(f"Thank you for returning the book!\n")
      else:
        self.books[book] = 1
        print(f"Thank you for the donation !\n")
      self.update_database("books")

    #get book from database
    def get_book(self, book_title):
      for book in self.books:
        if book_title == book.title and self.books[book] != 0:
          print(f"You're in luck, we have {self.books[book]} of these in stock !")
          return book
      print("HAHAHAH boo hoo ")
      return

    # borrow book from library
    def borrow_book(self, borrower_name, book_title):
      if self.book_exists(book_title):
        book = self.get_book(book_title)
        borrower = self.get_user(borrower_name)
        if(not borrower.borrowed_books.contains(book)):
          borrower.borrowed_books.append(book)
          self.books[book] -= 1
          self.update_database("books")
      else:
        print(f"either we dont have the book, or you already have it....\n")




Now that the library class is done, and we have all the necessary functions to manipulate the database, we create an interactive interface for the user to view, ask and update with the library

In [ ]:
# Instantiate the Library and more imports
library = Library()

# Widgets for adding a book
library_user_input = widgets.Text(placeholder='UserID', description='Library UserID:')
title_input = widgets.Text(placeholder='Book title', description='Title:')
author_input = widgets.Text(placeholder='Author name', description='Author:')
isbn_input = widgets.Text(placeholder='ISBN', description='ISBN:')
add_button = widgets.Button(description="Add Book")


# Widgets for searching a book
search_criteria = widgets.Dropdown(
    options=['Title', 'ISBN'],
    value='Title',
    description='Search by:'
)  # Added this line
search_input = widgets.Text(placeholder='value', description='Search :')
search_button = widgets.Button(description="Search Book")

# Widgets for viewing all books
view_button = widgets.Button(description="View All Books")

# Display output
output = widgets.Output()

# Function to add a book when the button is clicked
def on_search_button_clicked(b):
    with output:
        output.clear_output()
        if search_criteria.value == 'Title':
            library.get_book(search_input.value,)
        elif search_criteria.value == 'ISBN':
            library.get_book_by_isbn(search_input.value)


# Function to view all books
def on_view_button_clicked(b):
    with output:
        output.clear_output()
        library.view_all()

# Bind the buttons to their functions
add_button.on_click(on_add_button_clicked)
search_button.on_click(on_search_button_clicked)
view_button.on_click(on_view_button_clicked)

# Layout for adding a book
add_book_box = widgets.VBox([title_input, author_input, isbn_input, add_button])
search_book_box = widgets.HBox([search_criteria, search_input, search_button])
view_books_box = widgets.HBox([view_button])

# Display the interactive sections
display(add_book_box, search_book_box, view_books_box, output)

NameError: name 'on_add_button_clicked' is not defined